## Load historical IV

In [1]:
import math
import yfinance as yf

ticker = yf.Ticker('^VIX')
vix_df = ticker.history(period='15y', interval="1d")

## Helper functions

In [2]:
from scipy.stats import norm

def call_price(sp, st, r, t, vol, d):
    d1 = (math.log(sp/st) + (r - d + ((vol**2)/2))*t)/(vol*math.sqrt(t))
    d2 = d1 - vol*math.sqrt(t)
    return sp*math.exp(-d*t)*norm.cdf(d1) - st*math.exp(-r*t)*norm.cdf(d2)

def put_price(sp, st, r, t, vol, d):
    d1 = (math.log(sp/st) + (r - d + ((vol**2)/2))*t)/(vol*math.sqrt(t))
    d2 = d1 - vol*math.sqrt(t)
    return st*math.exp(-r*t)*norm.cdf(-d2) - sp*math.exp(-d*t)*norm.cdf(-d1)

def get_ticker_data(t):
    ticker = yf.Ticker(t) # GS-PD | FP.PA | MSFT | QQQ | SPY | QLD
    df = ticker.history(period='5y', interval="1d")
    df["vola"] = vix_df["Close"]
    return df[['Open', 'Close', 'Dividends', 'vola']].reset_index().to_numpy()



# Global params

In [34]:
OPT_CONT_SIZE = 100
init_cap = 1000
rate = 0.03
ticker = 'SPY'
streak_threshold = 3
change_threshold = 0.01

opt_duration = 3


In [35]:
series_data = get_ticker_data(ticker)

streak = 0
capital = init_cap
size = 0
opt_type = 0 # 0 call | 1 put
prev_change = 0
strike = 0

for idx, data in enumerate(series_data):
    date, open, close, divi, vola = data[0].date(), data[1], data[2], data[3], data[4]

    if size > 0 and strike > 0:
        if opt_type == 0:
            prem = call_price(close, strike, rate, opt_duration-1/365,vola/100,f_divi/close)*OPT_CONT_SIZE
            capital += size*prem
        if opt_type == 1:
            prem = put_price(close, strike, rate, opt_duration-1/365,vola/100,f_divi/close)*OPT_CONT_SIZE
            capital += size*prem
        size = 0
        strike = 0

    change = (close - open)/open
    if prev_change*change > 0 and abs(change) >= change_threshold:
        streak += 1
    else:
        streak = 0

    if streak >= streak_threshold:
        streak = 0
        # get futures dividend
        f_divi = 0
        for i in range(idx+1, min(idx+opt_duration+1, len(series_data))):
            if(series_data[i][2] != 0):
                f_divi = series_data[i][2]

        prem = 0
        if change > 0:
            opt_type = 1
            strike = open*0.95
            prem = put_price(close, strike, rate, opt_duration/365,vola/100,f_divi/close)*OPT_CONT_SIZE
        else:
            opt_type = 0
            strike = open*1.05
            prem = call_price(close, strike, rate, opt_duration/365,vola/100,f_divi/close)*OPT_CONT_SIZE

        size = math.floor(capital/prem)
        capital -= size*prem
        
    prev_change = change

        

print(capital)

0.5873789600589395
